<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/ReinforcementLearning/Breakout/DQN_atari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/

--2021-08-14 09:11:42--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11128004 (11M) [application/x-rar-compressed]
Saving to: ‘Roms.rar.1’

Roms.rar.1          100%[===================>]  10.61M   628KB/s    in 18s     

2021-08-14 09:12:00 (606 KB/s) - ‘Roms.rar.1’ saved [11128004/11128004]

mkdir: cannot create directory ‘/content/ROM/’: File exists

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Roms.rar


Would you like to replace the existing file /content/ROM/HC ROMS.zip
11826711 bytes, modified on 2019-12-22 11:24
with a new one
11826711 bytes, modified on 2019-12-22 11:24

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit 

In [ ]:
import random
import gym
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
EPOCHS = 1000
THRESHOLD = 10
MONITOR = False

In [ ]:
class DQN_atari():
    def __init__(self, env_string, batch_size=64, img_size=84, m=4):
        self.memory = deque(maxlen=100000)
        self.env = gym.make(env_string)
        input_size = env.observation_space.shape[0]
        action_size = env.action_space.n
        self.batch_size = batch_size
        self.gamma = 1.0
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.img_size = img_size
        self.m = m

        alpha = 0.01
        alpha_decay = 0.01
        

        self.model = Sequential()
        self.model.add(Conv2D(32, 8, (4,4), activation='relu', padding='valid', input_shape=(img_size, img_size, m)))
        self.model.add(Conv2D(64, 4, (2,2), activation='relu',padding='valid'))
        self.model.add(MaxPooling2D())
        self.model.add( Conv2D(64, 3, (1,1), activation='relu',padding='valid'))
        self.model.add(MaxPooling2D())
        self.model.add(Flatten())
        self.model.add(Dense(256, activation='elu'))
        self.model.add(Dense(action_size, activation='linear'))
        self.model.compile(loss='mse', optimizer=Adam(learning_rate=alpha, decay=alpha_decay))
        self.model_target = tf.keras.models.clone_model(self.model)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def choose_action(self, state, epsilon):
        if np.random.random() <= epsilon:
            return self.env.action_space.sample()
        else:
            return np.argmax(self.model.predict(state))


    #훈련에 필요한 중요한 이미지만 추출한다.
    def preprocess_state(self, img):
        img_temp = img[31:195]
        img_temp = tf.image.rgb_to_grayscale(img_temp)
        img_temp = tf.image.resize(img_temp, [self.img_size, self.img_size], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        img_temp = tf.cast(img_temp, tf.float32)
        return img_temp[:, :,0]

    def combine_images(self, img1, img2):
        if len(img1.shape) == 3 and img1.shape[0] == self.m:
            im = np.append(img1[1:, :, :], np.expand_dims(img2, 0), axis=2)
            return tf.expand_dims(im, 0)
        else:
            im = np.stack([img1]*self.m, axis=2)
            return tf.expand_dims(im, 0)
    
    def replay(self, batch_size):
        x_batch, y_batch = [], []
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            y_target = self.model_target.predict(state)
            y_target[0][action] = reward if done else reward + self.gamma * np.max(self.model.predict(next_state)[0])
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        self.model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)

    def train(self):
        scores = deque(maxlen=100)
        avg_score = []

        for e in range(EPOCHS):
            state = self.env.reset()
            state = self.preprocess_state(state)
            state = self.combine_images(state, state)
            done = False
            i = 0
            while not done:
                action = self.choose_action(state, self.epsilon)
                next_state,  reward, done, _ = self.env.step(action)
                next_state = self.preprocess_state(next_state)
                next_state = self.combine_images(next_state, state)
                self.remember(state, action, reward, next_state, done)
                state = next_state
                self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
                i += reward

            scores.append(i)
            mean_score = np.mean(scores) 
            avg_score.append(mean_score)
            if mean_score > THRESHOLD:
                print('solved after {} trials ✔'.format(e))
                return avg_score

            if e % 10 == 0:
                print('[Episode {}] - Average Score: {}.'.format(e, mean_score))
                self.model_target.set_weights(self.model.get_weights())

            self.replay(self.batch_size)

        print('Did not solve after {} episodes 😞'.format(e))
        return avg_scores

In [ ]:
tf.keras.backend.clear_session()
tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)
env_string = 'BreakoutDeterministic-v4'
agent = DQN_atari(env_string)
print("Main Model", agent.model.summary())
print("Target Model", agent.model_target.summary())
scores = agent.train()
plt.plot(scores)
plt.show()
agent.env.close()

In [ ]:
model.save('drive/MyDrive/Colab Notebooks/models/DQN_CartPole/CartPole-v0')